In [1]:
# from sklearn import svm,metrics
# from sklearn.neighbors import KNeighborsClassifier # 분류
# from sklearn.neighbors import KNeighborsRegressor # 회귀-예측
from sklearn.linear_model import LinearRegression # 선형회귀
from sklearn.preprocessing import PolynomialFeatures # 다항특성을 만들어주는 라이브러리
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False # 마이너스 표시 해결
# 한글설정
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # Windows 사용자
# matplotlib.rcParams['font.family'] = 'Apple Gothic' # Mac 사용자
matplotlib.rcParams['font.size'] = 10 # 글자크기

##### 데이터 피처 설명
- pclass : Passenger Class, 승객 등급
- survived : 생존 여부 : target 값이 됨.
- name : 승객 이름
- sex : 승객 성별
- age : 승객 나이
- sibsp : 탑승 한 형제/배우자 수
- parch : 탑승 한 부모/자녀 수
- ticket : 티켓 번호
- fare : 승객 지불 요금
- cabin : 선실 이름
- embarked : 승선항 (C = 쉘 부르그, Q = 퀸즈타운, S = 사우스 햄튼)
- body : 사망자 확인 번호 - 분석과정에서 제외해야 함. 머신러닝에 100% 영향을 미침.
- home.dest : 고향/목적지

In [153]:
# 타이타닉의 생존자 분류 모델을 구현하시오.
# target : survived
# data : 그외
# 데이터 확인 : nan,0, 타입,....
# 데이터 전처리 - nan,0,
# train,test세트
# 불필요한 피처를 제거 - 제거 피처는 체크 해 둘것
# 'name', 'ticket', 'body', 'cabin', 'home.dest' - name,cabin 사용 해 볼것.
# 각각 모델을 적용
# 정답률 체크
# 분류 : 로지스틱회귀, 결정트리, 랜덤포레스트
df_train = pd.read_csv('titanic_train.csv')
df_train.head(2)
df_test = pd.read_csv('titanic_test.csv')
df_train.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,2,1,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5,NaN,S,NaN,"England / Bennington, VT"
1,2,1,"Wells, Miss. Joan",female,4.0,1,1,29103,23.0,NaN,S,NaN,"Cornwall / Akron, OH"


In [108]:
df_train.head(2)

,pclass,survived,age,sibsp,parch,fare,embarked,sex_female,sex_male
0,2,1,13.0,0,1,19.5,S,True,False
1,2,1,4.0,1,1,23.0,S,True,False


In [31]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     393 non-null    int64  
 1   survived   393 non-null    int64  
 2   name       393 non-null    object 
 3   sex        393 non-null    object 
 4   age        305 non-null    float64
 5   sibsp      393 non-null    int64  
 6   parch      393 non-null    int64  
 7   ticket     393 non-null    object 
 8   fare       393 non-null    float64
 9   cabin      81 non-null     object 
 10  embarked   393 non-null    object 
 11  body       36 non-null     float64
 12  home.dest  218 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 40.0+ KB


In [56]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     393 non-null    int64  
 1   survived   393 non-null    int64  
 2   name       393 non-null    object 
 3   sex        393 non-null    object 
 4   age        305 non-null    float64
 5   sibsp      393 non-null    int64  
 6   parch      393 non-null    int64  
 7   ticket     393 non-null    object 
 8   fare       393 non-null    float64
 9   cabin      81 non-null     object 
 10  embarked   393 non-null    object 
 11  body       36 non-null     float64
 12  home.dest  218 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 40.0+ KB


In [160]:
df_train = df_train.fillna(0)
df_train.isnull().sum()

pclass        0
survived      0
age           0
sibsp         0
parch         0
fare          0
sex_male      0
embarked_Q    0
embarked_S    0
dtype: int64

In [161]:
df_test = df_test.fillna(0)
df_test.isnull().sum()

pclass        0
survived      0
age           0
sibsp         0
parch         0
fare          0
sex_male      0
embarked_Q    0
embarked_S    0
dtype: int64

In [158]:
df_train.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_male',
       'embarked_Q', 'embarked_S'],
      dtype='object')

In [159]:
df_test.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_male',
       'embarked_Q', 'embarked_S'],
      dtype='object')

In [8]:
df_train.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,393.000000,393.000000,305.000000,393.000000,393.000000,393.000000,36.00000
mean,2.300254,0.374046,29.030066,0.419847,0.417303,35.051854,167.75000
std,0.827724,0.484492,15.507318,0.807688,0.927970,60.662100,97.38154
min,1.000000,0.000000,0.330000,0.000000,0.000000,0.000000,18.00000
25%,2.000000,0.000000,19.000000,0.000000,0.000000,7.925000,79.50000
50%,3.000000,0.000000,27.000000,0.000000,0.000000,15.000000,169.00000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,30.500000,263.00000
max,3.000000,1.000000,80.000000,5.000000,9.000000,512.329200,327.00000


In [41]:
df_train.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,body,home.dest
0,3,0,"Rekic, Mr. Tido",male,38.0,0,0,349249,7.8958,NaN,S,NaN,NaN
1,2,1,"Phillips, Miss. Alice Frances Louisa",female,21.0,0,1,S.O./P.P. 2,21.0000,NaN,S,NaN,"Ilfracombe, Devon"


In [154]:
df_train = df_train.drop(['name','ticket','body','cabin','home.dest'],axis=1)

In [155]:
df_test = df_test.drop(['name','ticket','body','cabin','home.dest'],axis=1)

In [156]:
df_train = pd.get_dummies(df_train, columns = ['sex','embarked'],drop_first=True)

In [157]:
df_test = pd.get_dummies(df_test, columns = ['sex','embarked'],drop_first=True)

In [140]:
df_train

,pclass,survived,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,2,1,13.00,0,1,19.5000,True,False,False,False,True
1,2,1,4.00,1,1,23.0000,True,False,False,False,True
2,2,1,30.00,1,0,13.8583,True,False,True,False,False
3,3,0,NaN,0,0,7.7250,False,True,False,True,False
4,3,1,22.00,0,0,7.7250,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
911,3,1,0.17,1,2,20.5750,True,False,False,False,True
912,3,0,NaN,0,0,8.0500,False,True,False,False,True
913,3,1,NaN,0,0,7.7333,True,False,False,True,False
914,2,1,20.00,0,0,36.7500,True,False,False,False,True


In [162]:
data = df_train.drop('survived',axis=1)
target = df_train['survived']

In [163]:
train_input,test_input,train_target,test_target = train_test_split(
    data,target,test_size=0.2,random_state=42
)

In [120]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_scaled = ss.transform(train_input)
# test_scaled = ss.transform(test_input)

In [164]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_input,train_target)

c:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [165]:
print(lr.score(train_input,train_target))
print(lr.score(test_input,test_target))

0.7937158469945356
0.7663043478260869


In [166]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)

In [167]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=1,random_state=42)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9801919430588372 0.7869630043798341
